<a href="https://colab.research.google.com/github/GUNAPILLCO/neural_profit/blob/main/2_obtencion_preparacion_exploracion_datos/2_3_indicadores_tecnicos_retorno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2_3_Análisis de indicadores técnicos




## 0. Clonado de repositorio, importación de librerías y carga del dataset

### Clonado de repositorio e importación de librerías

In [1]:
#Clonamos el repo
#LINK DE REPOSITORIO: https://github.com/GUNAPILLCO/neural_profit

!git clone https://github.com/GUNAPILLCO/neural_profit.git

Cloning into 'neural_profit'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 166 (delta 50), reused 31 (delta 19), pack-reused 89 (from 1)
Receiving objects: 100% (166/166), 108.56 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Updating files: 100% (41/41), done.


In [34]:
import sys

#Instalación de librería pandas_market_calendars
#!{sys.executable} -m pip install -q pandas_market_calendars
#print("Librería instalada: pandas_market_calendars")

!{sys.executable} -m pip install -q ta
print("Librería instalada: technical-analysis")

# Utilidades generales
from datetime import datetime, timedelta
import os
import glob
import requests
import warnings
warnings.filterwarnings('ignore')

# Manejo y procesamiento de datos
import ta
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
# Calendario de mercados
#import pandas_market_calendars as mcal

from ta.momentum import StochasticOscillator, ROCIndicator
from ta.volatility import BollingerBands, AverageTrueRange

Librería instalada: technical-analysis


### Carga del dataset mnq_intraday_data

In [50]:
def load_df():
    """
    Función para cargar un archivo Parquet desde el repositorio clonado
    """
    # Definir la URL del archivo Parquet en GitHub
    df_path = '/content/neural_profit/2_obtencion_preparacion_exploracion_datos/mnq_intraday_data.parquet'

    # Leer el archivo Parquet y cargarlo en un DataFrame
    df = pd.read_parquet(df_path)

    # Asegurar que el índice esté en formato datetime
    df.index = pd.to_datetime(df.index)

    # Crear una nueva columna 'date' con la fecha extraída del índice
    df['date'] = df.index.date

    # Reordenar columnas: 'date', 'time_str', y luego el resto
    cols = ['date'] + [col for col in df.columns if col not in ['date']]

    df = df[cols]

    return df

In [51]:
mnq_intraday = load_df()

De las hipotesis aplicadas para calcular los alpha factor, el mejor target es el 'target_return_30':

In [52]:
def add_targets (df):
  #df['target_return_5'] = df.groupby('date')['close'].transform(lambda x: np.log(x.shift(-5)) - np.log(x))
  #df['target_return_15'] = df.groupby('date')['close'].transform(lambda x: np.log(x.shift(-15)) - np.log(x))
  df['target_return_30'] = df.groupby('date')['close'].transform(lambda x: np.log(x.shift(-30)) - np.log(x))
  return df

In [53]:
mnq_intraday = add_targets(mnq_intraday)
mnq_intraday

,date,open,high,low,close,volume,target_return_30
datetime,,,,,,,
2019-12-23 08:00:00-05:00,2019-12-23,8734.00,8734.25,8733.75,8733.75,31,0.000372
2019-12-23 08:01:00-05:00,2019-12-23,8734.00,8734.25,8733.75,8734.00,16,0.000286
2019-12-23 08:02:00-05:00,2019-12-23,8734.00,8734.00,8733.25,8733.25,23,0.000315
2019-12-23 08:03:00-05:00,2019-12-23,8734.25,8734.50,8734.00,8734.00,23,0.000315
2019-12-23 08:04:00-05:00,2019-12-23,8734.00,8734.00,8733.50,8733.75,12,0.000458
...,...,...,...,...,...,...,...
2025-06-13 15:56:00-04:00,2025-06-13,21624.50,21635.00,21613.50,21617.50,3251,NaN
2025-06-13 15:57:00-04:00,2025-06-13,21616.50,21635.25,21615.75,21623.75,2201,NaN
2025-06-13 15:58:00-04:00,2025-06-13,21623.25,21632.75,21616.50,21621.75,1859,NaN


## 1. Indicadores Técnicos

In [68]:
# Función para calcular los factores en un solo día
def calcular_indicadores_tecnicos(df_day: pd.DataFrame,
                    atributo: str,
                    ):
    # Se crea una copia del DataFrame del día para no modificar el original.
    df_day = df_day.copy()

    '''📊 RSI (Relative Strength Index): mide la fuerza relativa del precio en los últimos períodos. Oscila entre 0 y 100.
            Un valor alto indica sobrecompra, bajo indica sobreventa.
            Aquí se calcula sobre los precios de cierre del día actual.
            Es útil para detectar posibles puntos de reversión.
            '''
    df_day['rsi_14'] = ta.momentum.RSIIndicator(df_day[atributo]).rsi()
    df_day['rsi_7'] = ta.momentum.RSIIndicator(df_day[atributo], window=7).rsi()
    df_day['rsi_5'] = ta.momentum.RSIIndicator(df_day[atributo], window=5).rsi()
    df_day['rsi_3'] = ta.momentum.RSIIndicator(df_day[atributo], window=3).rsi()


    '''⚡ Momentum de 10 minutos: mide la variación porcentual entre el precio actual y el de hace 10 minutos.
            Si es positivo, hubo una suba reciente → posible continuación alcista.
            Si es negativo, indica presión bajista reciente.
        '''
    df_day['momentum_10'] = df_day[atributo].pct_change(10)
    df_day['momentum_5'] = df_day[atributo].pct_change(5)
    df_day['momentum_3'] = df_day[atributo].pct_change(3)

    '''📈 Relación de volumen actual contra su promedio de 20 minutos:
            Si >1: volumen superior al promedio reciente → podría reflejar interés o movimientos institucionales.
            Si <1: volumen bajo → señal de menor actividad o consolidación.
            Es una forma de capturar spikes de volumen sin usar el volumen en crudo.
        '''
    df_day['volume_ratio'] = df_day['volume'] / df_day['volume'].rolling(20).mean()

    '''🌀 MACD diferencial (señal de cruce): mide la diferencia entre la línea MACD y su señal (una media de sí misma).
            Cuando es positivo y creciente → momentum alcista.
            Cuando es negativo → momentum bajista.
            Muy usado para detectar giros de tendencia.
        '''
    df_day['macd'] = ta.trend.MACD(df_day[atributo]).macd_diff()

    '''📏 Distancia del precio actual a su EMA(20):
            Es un indicador de sobreextensión o retorno a la media.
            Si el precio está muy por encima de la EMA → posible reversión o aceleración.
            Este valor se expresa en forma de porcentaje relativo.
        '''
    df_day['price_ema20'] = df_day[atributo] / df_day[atributo].ewm(span=20).mean() - 1

# 🔄 Indicadores adicionales:

    # %K del Estocástico
    stoch = StochasticOscillator(df_day['high'], df_day['low'], df_day[atributo], window=14, smooth_window=3)
    df_day['stoch_k'] = stoch.stoch()

    # Bollinger Band %B
    bb = BollingerBands(close=df_day[atributo], window=20, window_dev=2)
    df_day['bb_percent'] = bb.bollinger_pband()

    # ATR normalizado
    atr = AverageTrueRange(high=df_day['high'], low=df_day['low'], close=df_day[atributo], window=14)
    df_day['atr'] = atr.average_true_range()
    df_day['atr_norm'] = df_day['atr'] / df_day[atributo]

    # ROC (Rate of Change)
    df_day['roc_5'] = ROCIndicator(close=df_day[atributo], window=5).roc()

    '''📅 Retorno futuro a 30 minutos (target para el modelo):
            Se calcula comparando el precio actual con el que tendrá 30 minutos más adelante.
            Es la variable objetivo que queremos predecir: si será positiva o negativa.
        '''
    return df_day

In [69]:
# Aplicar la función por día, respetando los cortes
mnq_intraday_target =  mnq_intraday.groupby('date').apply(calcular_indicadores_tecnicos, 'target_return_30')

#Esta línea aplica la función compute_factors a cada día de forma independiente:
#   mnq_intraday.groupby('date') divide el DataFrame en grupos por cada valor distinto en la columna 'date', la cual fue extraída del índice datetime.
#  .apply(compute_factors) ejecuta la función compute_factors sobre cada uno de esos días, calculando los indicadores técnicos y el retorno futuro sin cruzar datos entre días.
#  El resultado es un nuevo DataFrame que contiene todos los días concatenados, pero con un índice jerárquico (MultiIndex)

mnq_intraday_target.index = mnq_intraday_target.index.droplevel(0)  # Limpiamos índice jerárquico tras groupby
#Esta línea elimina el primer nivel del índice jerárquico que corresponde a la fecha del groupby.
# Después del groupby + apply, el índice queda en dos niveles: [date, datetime].
#Como ya tenés datetime en el índice y date como columna, este paso “aplana” el índice, dejando solo el índice horario original por minuto.


In [71]:
mnq_intraday_target

date      open      high  \
date       datetime                                                    
2019-12-23 2019-12-23 08:00:00-05:00  2019-12-23   8734.00   8734.25   
           2019-12-23 08:01:00-05:00  2019-12-23   8734.00   8734.25   
           2019-12-23 08:02:00-05:00  2019-12-23   8734.00   8734.00   
           2019-12-23 08:03:00-05:00  2019-12-23   8734.25   8734.50   
           2019-12-23 08:04:00-05:00  2019-12-23   8734.00   8734.00   
...                                          ...       ...       ...   
2025-06-13 2025-06-13 15:56:00-04:00  2025-06-13  21624.50  21635.00   
           2025-06-13 15:57:00-04:00  2025-06-13  21616.50  21635.25   
           2025-06-13 15:58:00-04:00  2025-06-13  21623.25  21632.75   
           2025-06-13 15:59:00-04:00  2025-06-13  21622.00  21632.25   
           2025-06-13 16:00:00-04:00  2025-06-13  21627.75  21656.25   

                                           low     close  volume  \
date       datetime                                                
2019-12-23 2019-12-23 08:00:00-05:00   8733.75   8733.75      31   
           2019-12-23 08:01:00-05:00   8733.75   8734.00      16   
           2019-12-23 08:02:00-05:00   8733.25   8733.25      23   
           2019-12-23 08:03:00-05:00   8734.00   8734.00      23   
           2019-12-23 08:04:00-05:00   8733.50   8733.75      12   
...                                        ...       ...     ...   
2025-06-13 2025-06-13 15:56:00-04:00  21613.50  21617.50    3251   
           2025-06-13 15:57:00-04:00  21615.75  21623.75    2201   
           2025-06-13 15:58:00-04:00  21616.50  21621.75    1859   
           2025-06-13 15:59:00-04:00  21618.00  21628.00    2695   
           2025-06-13 16:00:00-04:00  21622.50  21651.75    6507   

                                      target_return_30     rsi_14      rsi_7  \
date       datetime                                                            
2019-12-23 2019-12-23 08:00:00-05:00          0.000372        NaN        NaN   
           2019-12-23 08:01:00-05:00          0.000286        NaN        NaN   
           2019-12-23 08:02:00-05:00          0.000315        NaN        NaN   
           2019-12-23 08:03:00-05:00          0.000315        NaN        NaN   
           2019-12-23 08:04:00-05:00          0.000458        NaN        NaN   
...                                                ...        ...        ...   
2025-06-13 2025-06-13 15:56:00-04:00               NaN  63.154131  77.443346   
           2025-06-13 15:57:00-04:00               NaN  63.154131  77.443346   
           2025-06-13 15:58:00-04:00               NaN  63.154131  77.443346   
           2025-06-13 15:59:00-04:00               NaN  63.154131  77.443346   
           2025-06-13 16:00:00-04:00               NaN  63.154131  77.443346   

                                          rsi_5  ...  momentum_5  momentum_3  \
date       datetime                              ...                           
2019-12-23 2019-12-23 08:00:00-05:00        NaN  ...         NaN         NaN   
           2019-12-23 08:01:00-05:00        NaN  ...         NaN         NaN   
           2019-12-23 08:02:00-05:00        NaN  ...         NaN         NaN   
           2019-12-23 08:03:00-05:00        NaN  ...         NaN   -0.153846   
           2019-12-23 08:04:00-05:00  78.587326  ...         NaN    0.599908   
...                                         ...  ...         ...         ...   
2025-06-13 2025-06-13 15:56:00-04:00  84.773777  ...         0.0    0.000000   
           2025-06-13 15:57:00-04:00  84.773777  ...         0.0    0.000000   
           2025-06-13 15:58:00-04:00  84.773777  ...         0.0    0.000000   
           2025-06-13 15:59:00-04:00  84.773777  ...         0.0    0.000000   
           2025-06-13 16:00:00-04:00  84.773777  ...         0.0    0.000000   

                                      volume_ratio          macd  price_ema20  \
date       datetime                                        

##2. Calculo de Information Coefficient (IC) por día

El Information Coefficient (IC) es la correlación de Spearman entre el factor y el retorno futuro. Mide si el ranking del factor predice el ranking del retorno.

🔹 IC > 0: el factor tiene relación positiva → valores altos del factor tienden a preceder retornos positivos.

🔹 IC < 0: relación inversa → valores altos del factor tienden a preceder retornos negativos (reversión).

🔹 IC ≈ 0: el factor es probablemente ruido o no tiene señal útil.

Valores comunes en finanzas:

🔹 0.05 a 0.1: débil pero útil.

🔹 >0.1: muy raro, pero fuerte.

🔹 <0.05 o cercano a 0: probablemente ruido.

In [72]:
from scipy.stats import spearmanr
import numpy as np

def calcular_IC (df: pd.DataFrame,
                    target: str,
                    ):
  factors = ['rsi_14', 'rsi_7', 'rsi_5', 'rsi_3', 'momentum_10', 'momentum_5', 'momentum_3', 'volume_ratio', 'macd', 'price_ema20', 'stoch_k', 'bb_percent', 'atr_norm', 'roc_5', 'body_ratio']
    #target = 'future_return_5min'

  # Agrupar por día
  grouped = df.groupby('date')

  print("📊 IC promedio por factor (diario):\n")

  for factor in factors:
      daily_ics = []

      for date, group in grouped:
          df_valid = group[[factor, target]].dropna()
          if len(df_valid) >= 20:  # al menos 20 datos para evitar ruido
              ic = spearmanr(df_valid[factor], df_valid[target]).correlation
              if np.isfinite(ic):
                  daily_ics.append(ic)

      if daily_ics:
          mean_ic = np.mean(daily_ics)
          std_ic = np.std(daily_ics)
          print(f"{factor:15}: IC medio = {mean_ic:.4f} | std = {std_ic:.4f} | días válidos = {len(daily_ics)}")
      else:
          print(f"{factor:15}: sin días válidos para cálculo")

In [77]:
calcular_IC(mnq_intraday_close, 'close')

📊 IC promedio por factor (diario):

rsi_14         : IC medio = 0.3347 | std = 0.1771 | días válidos = 1311
rsi_7          : IC medio = 0.2508 | std = 0.1264 | días válidos = 1311
rsi_5          : IC medio = 0.2156 | std = 0.1063 | días válidos = 1311


KeyboardInterrupt: 

In [78]:
calcular_IC(mnq_intraday_target, 'target_return_30')

ValueError: 'date' is both an index level and a column label, which is ambiguous.

📈 Interpretación de los resultados — Information Coefficient (IC)

El **Information Coefficient (IC)** mide la correlación de Spearman entre el valor del factor y el retorno futuro. Es decir, evalúa si el ranking del factor anticipa el ranking del rendimiento posterior.

| 🏅 Top factores con mejor IC | IC Medio | Interpretación práctica                                        |
| ---------------------------- | -------- | -------------------------------------------------------------- |
| **rsi\_14**                  | -0.0534  | Señal clara de reversión. Es el más consistente del conjunto.  |
| **price\_ema20**             | -0.0435  | Reversión hacia la media. Buena señal complementaria.          |
| **bb\_percent**              | -0.0387  | Similar a RSI, sobrecompra/sobreventa. Señal confiable.        |
| **stoch\_k**                 | -0.0374  | Confirma reversión; alternativa válida al RSI.                 |
| **atr\_norm**                | +0.0471  | Único con IC positivo fuerte. Alta volatilidad → continuación. |
| **rsi\_7**                   | -0.0405  | Más sensible que RSI(14), mantiene coherencia.                 |

**Estos pueden capturar efectos de reversión o momentum en condiciones específicas, y probablemente rindan mejor combinados en modelos no lineales (XGBoost, LSTM, etc.).**


📉 Factores con señal débil o nula:

| Factor                       | IC Medio      | Nota                                       |
| ---------------------------- | ------------- | ------------------------------------------ |
| momentum_3/5/10, roc_5   | -0.02 a -0.03 | Señales débiles, sin aporte significativo. |
| macd`                       | -0.0196       | Muy débil para este timeframe.             |
| volume_ratio, body_ratio | ≈ 0           | No aportan valor predictivo.               |



Filtro el dataset para quedarme solo con las factores que aportarían valor a mi modelo:

In [ ]:
# Lista de columnas útiles
keep_cols = [
    'open', 'high', 'low', 'close', 'volume', 'date',
    'rsi_14', 'rsi_7', 'price_ema20', 'bb_percent',
    'stoch_k', 'atr_norm',
    'future_return_5min'
]


# Filtrar el dataset
mnq_intraday_clean = mnq_intraday_factors[keep_cols].dropna()

📌 Notas:

- dropna() elimina las filas con NaN, que suelen ser las primeras del día por el cálculo de ventanas (RSI, ATR, etc.).
- Esto asegura que el dataset esté limpio para entrenamiento de modelos.

In [ ]:
mnq_intraday_clean

##### 2.2. IDEAS PARA FEATURES

🧠 Ideas para combinar:

- Filtrar señales por volatilidad: usar atr_norm como feature auxiliar para condicionar otras señales.
- Ensemble de reversión: combinar rsi, stoch_k, bb_percent en un score compuesto.
- Interacciones: por ejemplo, si rsi está bajo y bb_percent < 0.1 → señal de rebote más fuerte.

###### ✅ 1. reversal_score: Ensemble de reversión suave

Combina señales de sobreventa (RSI, %K, BB) en una puntuación continua. Cuanto más baja, mayor potencial de rebote.

In [ ]:
# Escalar a rango 0-1 donde corresponda
mnq_intraday_clean['rsi_norm'] = mnq_intraday_clean['rsi_14'] / 100
mnq_intraday_clean['stoch_k_norm'] = mnq_intraday_clean['stoch_k'] / 100
# bb_percent ya está entre 0-1

# Reversal Score: promedio de los tres, invertido
mnq_intraday_clean['reversal_score'] = 1 - (
    mnq_intraday_clean['rsi_norm'] +
    mnq_intraday_clean['stoch_k_norm'] +
    mnq_intraday_clean['bb_percent']
) / 3

###### ✅ 2. volatility_filter: Filtro de condiciones de mercado
Crea una feature binaria que indica si estamos en un contexto de alta volatilidad según ATR normalizado.

In [ ]:
mnq_intraday_clean['volatility_filter'] = (mnq_intraday_clean['atr_norm'] > 0.02).astype(int)

El umbral 0.02 es empírico — podés ajustarlo con histogramas o percentiles.

###### ✅ 3. rebound_trigger: Señal fuerte combinada (regla lógica)

Detecta eventos donde:

- RSI está bajo
- %B está por debajo de 0.1 (precio muy fuera de banda)
- Volatilidad es alta

In [ ]:
mnq_intraday_clean['rebound_trigger'] = (
    (mnq_intraday_clean['rsi_14'] < 30) &
    (mnq_intraday_clean['bb_percent'] < 0.1) &
    (mnq_intraday_clean['atr_norm'] > 0.02)
).astype(int)


In [ ]:
mnq_intraday_clean

## 2.3. ANÁLISIS POR CUANTILES

¿Qué hace este código?

Para cada factor:

- Se divide cada día en 5 quintiles (qcut).
- Calcula el retorno medio futuro de cada quintil.
- Acumula los resultados de todos los días válidos.
- Al final, te da el retorno promedio por quintil en todo el dataset.

Esto te permite ver si, por ejemplo, los valores bajos del RSI (Q0) tienden a tener retornos positivos → reversión. O si Q4 tiene retorno positivo → momentum.

In [ ]:
# Factores seleccionados que querés evaluar
factors = ['rsi_14', 'rsi_7', 'price_ema20', 'bb_percent', 'stoch_k', 'atr_norm', 'reversal_score']

target = 'future_return_5min'

# Agrupamos por día
grouped = mnq_intraday_clean.groupby('date')

print("\n📈 Retorno promedio por quintil (promedio diario):\n")

for factor in factors:
    all_quantile_returns = []

    for date, group in grouped:
        df_valid = group[[factor, target]].dropna()

        if len(df_valid) >= 30:
            try:
                df_valid['q'] = pd.qcut(df_valid[factor], 5, labels=False, duplicates='drop')
                mean_ret = df_valid.groupby('q')[target].mean()

                if len(mean_ret) == 5:
                    all_quantile_returns.append(mean_ret.values)
            except ValueError:
                continue  # Si no hay suficientes valores únicos para dividir en quintiles

    if all_quantile_returns:
        avg_qrets = np.mean(all_quantile_returns, axis=0)
        print(f"\n🔹 {factor} – retorno medio por quintil (Q0 a Q4):")
        for i, r in enumerate(avg_qrets):
            print(f"  Q{i}: {r:.5f}")
    else:
        print(f"\n🔹 {factor}: sin suficientes datos diarios para quintiles")

### 📊 Retorno promedio por quintil (Q0 a Q4)


| Factor           | Q0      | Q1      | Q2      | Q3      | Q4       | Interpretación                                               |
|------------------|---------|---------|---------|---------|----------|--------------------------------------------------------------|
| **rsi_14**        | 0.00008 | 0.00001 | 0.00000 | -0.00000| -0.00006 | Reversión clara: Q0 positivo, Q4 negativo                    |
| **rsi_7**         | 0.00006 | 0.00001 | 0.00000 | -0.00000| -0.00005 | Reversión leve, misma estructura que RSI_14                 |
| **price_ema20**   | 0.00008 | 0.00000 | -0.00001| -0.00000| -0.00004 | Sobreextensión a la media sugiere reversión                 |
| **bb_percent**    | 0.00006 | 0.00002 | 0.00000 | -0.00001| -0.00004 | Precio cerca del borde inferior tiende a rebotar            |
| **stoch_k**       | 0.00006 | 0.00002 | 0.00000 | -0.00001| -0.00004 | Señal alternativa de reversión como RSI y BB                |
| **atr_norm**      | -0.00005| -0.00001| 0.00001 | 0.00000 | 0.00008  | Alta volatilidad favorece continuación                      |
| **reversal_score**| -0.00005| -0.00001| -0.00000| 0.00002 | 0.00006  | Score compuesto: buena separación entre extremos            |


✅ Resumen conceptual

Los valores reflejan el retorno promedio futuro a 5 minutos, agrupado por quintiles del factor. Es decir:

- Q0 = valores más bajos del factor (ej: RSI más bajo)
- Q4 = valores más altos del factor

### 📊 Conclusiones clave por tipo de factor


#### 🔄 Factores de reversión (rsi_14, rsi_7, price_ema20, bb_percent, stoch_k)

Patrón observado:
Q0 tiene retorno positivo, Q4 negativo → clara señal de reversión estadística

Por ejemplo: rsi_14

  Q0: +0.00008 → RSI muy bajo → sube
  
  Q4: -0.00006 → RSI muy alto → baja

✅ Interpretación:
Los valores extremos (muy bajos) anticipan rebotes, y los valores altos anticipan caídas, aunque el efecto sea leve.

#### 🔊 Factor de volatilidad (atr_norm)


Patrón:
Q4 tiene retorno positivo → alta volatilidad favorece continuación

Ejemplo:

atr_norm

  Q0: -0.00005 → baja volatilidad → mercado plano

  Q4: +0.00008 → alta volatilidad → rompe en una dirección


✅ Interpretación:
Este factor no predice dirección por sí solo, pero ayuda a filtrar cuándo las señales funcionan mejor (alta volatilidad = movimiento significativo).

#### 🧠 Feature combinada reversal_score


Patrón observado: Q0 negativo, Q4 positivo → signo invertido por construcción

Y se comporta bien: Q4 = +0.00006

✅ Interpretación:

Confirma que el score compuesto refleja la lógica de reversión. Es más fácil de interpretar para un modelo ML y combina múltiples señales.

### 🎯 Recomendación general

Los factores muestran consistencia en su estructura predictiva: reversión suave, efecto de volatilidad, combinación útil.

Aunque los valores absolutos de retorno son pequeños (esperado en timeframe de 1 minuto), la dirección del efecto está clara y sistemática, lo cual es valioso para modelos de ML.

Estas señales pueden ser muy efectivas en modelos que aprovechan la combinación y no linealidad (ej. XGBoost, LSTM).

## ESTABILIDAD DE ALPHA FACTORS?


In [ ]:
mnq_intraday_clean

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Suponemos que ya cargaste mnq_intraday_clean con estos factores
alpha_factors = ['rsi_14', 'rsi_7', 'price_ema20', 'bb_percent', 'stoch_k', 'atr_norm', 'reversal_score']
target = 'future_return_5min'

# Agrupar por día
grouped = mnq_intraday_clean.groupby('date')

# Calcular IC diario
ic_diarios = {factor: [] for factor in alpha_factors}
fechas = []

for date, group in grouped:
    fechas.append(date)
    for factor in alpha_factors:
        df_valid = group[[factor, target]].dropna()
        if len(df_valid) >= 30:
            ic = spearmanr(df_valid[factor], df_valid[target]).correlation
        else:
            ic = np.nan
        ic_diarios[factor].append(ic)

# Convertir a DataFrame de IC diarios
df_ic = pd.DataFrame(ic_diarios, index=pd.to_datetime(fechas))

# Graficar la media móvil de 20 días
plt.figure(figsize=(14, 6))
for factor in alpha_factors:
    df_ic[factor].rolling(20).mean().plot(label=factor)

plt.title("📈 IC promedio móvil (20 días) por alpha factor")
plt.xlabel("Fecha")
plt.ylabel("Information Coefficient (Spearman)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()